In [1]:
import autograd.numpy as np
from autograd import grad, hessian, jacobian
from scipy.optimize import minimize

In [2]:
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *

username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_subset_20200304.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
data_dict['R_hat'][reg_2_num['usa'], 3] = 2.
data_dict['R_hat'][reg_2_num['usa'], 4] = 2.



X_0 = np.ones(n*g + n)*0.5


eval_F = lambda x: reduced_counterfactual_overdetermined(x, data_dict)
eval_f = lambda x: np.sum(eval_F(x)**2)


eval_grad_f = grad(eval_f)
bounds_f = [(1e-14, None) for x in range(n*g+n)]

In [10]:
sol = minimize(eval_f, X_0*2, jac=eval_grad_f, tol=1e-9, method='L-BFGS-B', bounds=bounds_f, options={'maxiter':15000})

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


In [11]:
sol.x

array([0.99670496, 1.00338663, 1.00272572, 0.90824538, 0.92482971,
       0.99717422, 0.99411464, 1.00075034, 1.00290584, 0.91213538,
       0.99855759, 0.98983496, 0.99726416, 1.00028354, 1.00214201,
       0.91348369, 0.85215626, 0.9964438 , 1.00276212, 1.00477649,
       1.00250193, 0.89424673, 0.95586227, 1.00655292, 0.99487205,
       1.00525945, 0.99948564, 0.96726581, 0.90408466, 0.97663564,
       1.00838621, 1.00868654, 1.0070361 , 0.88429254, 0.90014724,
       1.00716476, 0.99890447, 1.00133037, 1.00251474, 0.91427347,
       0.93839804, 0.99373382, 0.97064927, 0.99026164, 0.99690873,
       0.92109431, 0.87525209, 0.99678465, 0.99816701, 0.99861313,
       1.00540921, 0.86782113, 0.93238506, 0.99116745, 0.98512934,
       0.99571713, 0.99048084, 0.90288501, 0.96482506, 0.99443779,
       0.96675057, 0.99537597, 0.99957451, 0.89675107, 0.87770725,
       0.96380424, 0.99761985, 1.00086679, 1.00115784, 0.91436421,
       0.93228433, 0.99755523, 0.98708368, 0.9830276 , 0.99626

In [12]:
sol.success

True

In [13]:
sol

      fun: 0.0002536942385779576
 hess_inv: <938x938 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.44472235e-05, -9.62194000e-06, -6.32120331e-05, -6.64761843e-06,
       -4.41528442e-06, -9.50767519e-06,  2.16243295e-06, -4.20662117e-06,
        9.11643050e-06, -1.27996275e-05,  4.74829350e-06,  1.41488362e-06,
        6.90903886e-06,  4.99864038e-07,  8.39511033e-06,  4.52121083e-06,
       -1.81127264e-05, -1.27894359e-05, -8.01309457e-06,  1.31011373e-05,
        1.89026874e-06, -2.94062508e-05,  1.37903534e-05, -1.25161997e-05,
        1.45273675e-05,  7.16076004e-06,  2.59801825e-05, -4.81663218e-06,
       -2.12390967e-05,  4.68296828e-06,  1.51017018e-05,  5.44313742e-06,
       -2.08629692e-05,  2.72929947e-05, -1.30120910e-05, -3.01962164e-06,
        8.80174805e-06,  4.11216737e-06,  1.37908685e-05,  5.90420718e-05,
       -8.96387487e-06,  3.30504395e-06,  5.20890470e-06, -1.29903027e-06,
       -5.88450948e-06, -4.38405902e-07,  3.92867327e-05,  1.12989055e-

In [8]:
## Seems like constraining the length of w_hat and r_hat works! Try to justify this later!

In [7]:
r_hat_star, w_hat_star = get_values_from_X_reduced(sol.x, data_dict)

In [8]:
E_hat_star = get_E_hat(w_hat_star, r_hat_star, data_dict)

In [9]:
E_hat_star.sum()

133.03857954476084